In [1]:
# @inproceedings{karkkainenfairface,
#   title={FairFace: Face Attribute Dataset for Balanced Race, Gender, and Age for Bias Measurement and Mitigation},
#   author={Karkkainen, Kimmo and Joo, Jungseock},
#   booktitle={Proceedings of the IEEE/CVF Winter Conference on Applications of Computer Vision},
#   year={2021},
#   pages={1548--1558}
# }

In [82]:
import pandas as pd
import dlib

In [118]:
fairface_train = pd.read_csv("fairface_label_train.csv")
fairface_val = pd.read_csv("fairface_label_val.csv")

In [134]:
# I'm sorry for doing this
def give_filenames(dataset):
    male_state = (dataset['age'] == '20-29') & (dataset['race'] == 'White') & (dataset['gender'] == 'Male') 
    female_state = (dataset['age'] == '20-29') & (dataset['race'] == 'White') & (dataset['gender'] == 'Female') 
    male = dataset[male_state][:301]
    female = dataset[female_state][:301]
    
    male_filenames = []
    for i in range(len(male)):
        male_filenames.append(male.iloc[i]['file'])
        
    female_filenames = []
    for i in range(len(female)):
        female_filenames.append(female.iloc[i]['file'])
    return male_filenames, female_filenames

## For Training Dataset

In [135]:
# Split training dataset to male and female

# import os
# import shutil

# for file in male_filenames:
#     filename_from = os.path.join('fairface-img-margin025-trainval/', file)
#     filename_to = os.path.join('male/', file[6:])
# #     os.rename(filename_from, filename_to)
#     if (os.path.exists(filename_from)):
#         shutil.move(filename_from, filename_to)
#         os.replace(filename_from, filename_to)
#         filein.close()

## For Testing Dataset

In [142]:
# male_fn, female_fn = give_filenames(fairface_val)

In [148]:
# Split val dataset to male and female

# import os
# import shutil

# for file in female_fn:
#     filename_from = os.path.join('fairface-img-margin025-trainval/', file)
#     filename_to = os.path.join('female/', file[4:])
# #     os.rename(filename_from, filename_to)
#     if (os.path.exists(filename_from)):
#         shutil.move(filename_from, filename_to)
#         os.replace(filename_from, filename_to)
#         filein.close()